In [4]:
import cv2
import numpy as np
import json

In [8]:
def find_highlighted_pixels(image_path, threshold=200):
    # Load the image
    image = cv2.imread(image_path)

    # Convert the image to grayscale
    grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to identify the highlighted areas
    _, binary_image = cv2.threshold(grayscale_image, threshold, 255, cv2.THRESH_BINARY)

    # Find contours in the binary image
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Extract pixel coordinates from contours
    highlighted_pixels = []
    for contour in contours:
        for point in contour:
            highlighted_pixels.append(tuple(point[0]))

    return highlighted_pixels

def save_pixels_to_json(pixel_coordinates, json_file_path):

    pixel_coordinates = [tuple(map(int, coord)) for coord in pixel_coordinates]
    data = {"highlighted_pixels": pixel_coordinates}
    with open(json_file_path, 'w') as json_file:
        json.dump(data, json_file, indent=2)

In [10]:
image_path = "/content/drive/MyDrive/noise_rating_modelling/freeway_map_california.jpg"
pixels = find_highlighted_pixels(image_path=image_path)
save_pixels_to_json(pixels, "/content/drive/MyDrive/noise_rating_modelling/highway.json")

In [12]:
import json

def create_binary_matrix(json_file_path, output_json_file_path):
    # Load existing JSON data
    with open(json_file_path, 'r') as json_file:
        data = json.load(json_file)

    # Extract highlighted pixel coordinates
    highlighted_pixels = data.get("highlighted_pixels", [])

    # Find the dimensions of the matrix
    max_row = max(coord[0] for coord in highlighted_pixels)
    max_col = max(coord[1] for coord in highlighted_pixels)

    # Initialize a matrix with zeros
    binary_matrix = [[0] * (max_col + 1) for _ in range(max_row + 1)]

    # Set the corresponding elements to 1
    for coord in highlighted_pixels:
        binary_matrix[coord[0]][coord[1]] = 1

    # Create a new JSON file with the binary matrix
    new_data = {"binary_matrix": binary_matrix}
    with open(output_json_file_path, 'w') as output_json_file:
        json.dump(new_data, output_json_file, indent=2)


# Example usage
input_json_path = "/content/drive/MyDrive/noise_rating_modelling/highway.json"
output_json_path = "/content/drive/MyDrive/noise_rating_modelling/highway_binary.json"
create_binary_matrix(input_json_path, output_json_path)
